In [1]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model

import os

In [2]:

class QuadDense(Layer):
    def __init__(self, units=64, activation = 'relu'):
        super(QuadDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        w1_init = tf.random_normal_initializer()
        self.w1 = tf.Variable(name = 'kernel_1', initial_value = w1_init(shape = (input_shape[-1], self.units), dtype = 'float32'))
        
        w2_init = tf.random_normal_initializer()
        self.w2 = tf.Variable(name = 'kernel_2', initial_value = w2_init(shape = (input_shape[-1], self.units), dtype = 'float32'))

        bias_init = tf.zeros_initializer()
        self.b = tf.Variable(name = 'bias', initial_value = bias_init(shape = (self.units,), dtype = 'float32'))


    def call(self, inputs):
        return self.activation((tf.matmul(tf.math.square(inputs), self.w1)+tf.matmul(inputs, self.w2)+self.b))

                             


In [3]:
class ResidualBlock(Model):
    def __init__(self, n_filters, kernel_size):
        super(ResidualBlock, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(kernel_size=kernel_size, filters = n_filters, padding = 'same')
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(kernel_size = kernel_size, filters = n_filters, padding = 'same')
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.activation = tf.keras.layers.Activation('relu')

        self.conv_adjuster = tf.keras.layers.Conv2D(kernel_size= (1,1), filters = n_filters, padding = 'same')

        self.add = tf.keras.layers.Add()

    
    def call(self, inputs):
        print(inputs.shape)
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation(x)

        input_tensor = self.conv_adjuster(inputs)
        x = self.add([x, input_tensor])
        x = self.activation(x)

        return x


In [4]:
class InceptionBlock(Model):
    def __init__(self, n_filters):
        super(InceptionBlock, self).__init__()
        self.path1_conv1 = tf.keras.layers.Conv2D(kernel_size = (1,1), filters = n_filters, padding = 'same')
        self.path1_conv2 = tf.keras.layers.Conv2D(kernel_size= (3,3), filters = n_filters, padding = 'same')
        self.path1_bn = tf.keras.layers.BatchNormalization()


        self.path2_conv1 = tf.keras.layers.Conv2D(kernel_size = (1,1), filters = n_filters*2, padding = 'same')
        self.path2_conv2 = tf.keras.layers.Conv2D(kernel_size= (3,3), filters = n_filters*2, padding = 'same')
        self.path2_bn = tf.keras.layers.BatchNormalization()

        self.path3_conv1 = tf.keras.layers.Conv2D(kernel_size = (1,1), filters = n_filters//2, padding = 'same')
        self.path3_conv2 = tf.keras.layers.Conv2D(kernel_size= (5,5), filters = n_filters//2, padding = 'same')
        self.path3_bn = tf.keras.layers.BatchNormalization()

        self.path4_pool = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = (1,1), padding='same')
        self.path4_conv1 = tf.keras.layers.Conv2D(kernel_size = (1,1), filters = n_filters, padding = 'same')
        self.path4_bn = tf.keras.layers.BatchNormalization()

        self.activation = tf.keras.layers.Activation('relu')

        self.concat = tf.keras.layers.Concatenate()

    
    def call(self, inputs):
        x1 = self.path1_conv1(inputs)
        x1 = self.path1_conv2(x1)
        x1 = self.path1_bn(x1)
        x1 = self.activation(x1)

        x2 = self.path2_conv1(inputs)
        x2 = self.path2_conv2(x2)
        x2 = self.path2_bn(x2)
        x2 = self.activation(x2)

        x3 = self.path3_conv1(inputs)
        x3 = self.path3_conv2(x3)
        x3 = self.path3_bn(x3)
        x3 = self.activation(x3)

        x4 = self.path4_pool(inputs)
        x4 = self.path4_conv1(x4)
        x4 = self.path4_bn(x4)
        x4 = self.activation(x4)

        outputs = self.concat([x1, x2, x3, x4])

        return outputs

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train", x_train.shape)
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(( 28, 28, 1)),
  InceptionBlock(n_filters=64),
  tf.keras.layers.Conv2D(kernel_size= (3,3), filters = 32),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

x_train (60000, 28, 28)
Epoch 1/5
1875/1875 [==============================] - 121s 62ms/step - loss: 0.4356 - accuracy: 0.9335
Epoch 2/5
1875/1875 [==============================] - 118s 63ms/step - loss: 0.1430 - accuracy: 0.9673
Epoch 3/5
1875/1875 [==============================] - 78s 42ms/step - loss: 0.0758 - accuracy: 0.9790
Epoch 4/5
1875/1875 [==============================] - 137s 73ms/step - loss: 0.0615 - accuracy: 0.9805
Epoch 5/5
1875/1875 [==============================] - 96s 51ms/step - loss: 0.0508 - accuracy: 0.9845


In [6]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 5s 15ms/step - loss: 0.0505 - accuracy: 0.9837


[0.05054245889186859, 0.9836999773979187]

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_block (InceptionB  (None, 28, 28, 288)      211872    
 lock)                                                           
                                                                 
 conv2d_7 (Conv2D)           (None, 26, 26, 32)        82976     
                                                                 
 flatten (Flatten)           (None, 21632)             0         
                                                                 
 dropout (Dropout)           (None, 21632)             0         
                                                                 
 dense (Dense)               (None, 10)                216330    
                                                                 
Total params: 511,178
Trainable params: 510,602
Non-trainable params: 576
________________________________________________